In [124]:
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns



import warnings
warnings.filterwarnings("ignore")


In [3]:


pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
# This is not needed if chromedriver is already on your path:
chromedriver_path = "/Users/natalia/Downloads/chromedriver_mac64/chromedriver"

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--verbose")
# options.add_argument("--headless")

driver = webdriver.Chrome(options=options, executable_path=chromedriver_path)
driver.get("https://www.booking.com/searchresults.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4AQfIAQzYAQPoAQH4AQOIAgGoAgO4At_b5aAGwAIB0gIkMjlmODkyN2EtYTczMi00Y2IxLWFiMDAtMDEwMjhiNmUzYWM52AIE4AIB&aid=397594&ss=Costa+Dorada&ssne=Costa+Dorada&ssne_untouched=Costa+Dorada&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=1389&dest_type=region&checkin=2023-07-03&checkout=2023-07-09&group_adults=2&no_rooms=1&group_children=2&age=4&age=8&sb_travel_purpose=leisure&offset=1500")

/var/folders/fy/wscq04p17_b8_9hgdslfplgh0000gn/T/ipykernel_2963/511819271.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=chromedriver_path)


### El siguiente web scraping de la web de Booking.com se realizo el día sábado 25 de marzo de 2023.
El web scraping es una técnica utilizada para extraer información de sitios web. En el caso de Selenium, es una herramienta de automatización de pruebas que también se puede utilizar para el web scraping.

In [64]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Configuración del webdriver
driver = webdriver.Chrome() 

# Definición del número de páginas
num_pages = 40

# Navega al URL Booking.com 
offset = 0
bookingUrl = 'https://www.booking.com/searchresults.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4AQfIAQzYAQPoAQH4AQOIAgGoAgO4At_b5aAGwAIB0gIkMjlmODkyN2EtYTczMi00Y2IxLWFiMDAtMDEwMjhiNmUzYWM52AIE4AIB&aid=397594&ss=Costa+Dorada&ssne=Costa+Dorada&ssne_untouched=Costa+Dorada&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=1389&dest_type=region&checkin=2023-07-03&checkout=2023-07-09&group_adults=2&no_rooms=1&group_children=2&age=4&age=8&sb_travel_purpose=leisure'
driver.get(bookingUrl)

# Tiempo de espera a que se carguen los resultados de la búsqueda
time.sleep(1)

# Configuración de una lista para almacenar los datos 
hotel_data = []

# Recorrido del número especificado de páginas y extracción los datos 
for page in range(num_pages):

    # Obtención de los datos del hotel en la página actual
    hotel_elements = driver.find_elements(By.XPATH, '//div[@data-testid="property-card"]')
    for index, hotel_element in enumerate(hotel_elements):
        
        # Nombre del hotel
        name_element = hotel_element.find_elements(By.XPATH, '//div[@data-testid="title"]')[index]
        name = name_element.text
        
        # Precio del hotel
        price_element = hotel_element.find_elements(By.XPATH, '//span[@data-testid="price-and-discounted-price"]')[index]
        price = price_element.text
        
        # Puntuación de la reseña del hotel
        score_element = hotel_element.find_elements(By.XPATH, '//div[@class="b5cd09854e d10a6220b4"]')
        
        # Ciudad
        address_element = hotel_element.find_elements(By.XPATH, '//span[@data-testid="address"]')[index]
        address = address_element.text
        
        # Tipo de alojamiento
        tipo_alojamiento_element = hotel_element.find_elements(By.XPATH,'//span[@class="df597226dd"]')[index]
        tipo_alojamiento = tipo_alojamiento_element.text
        
        # Política de cancelación
        cancelacion_element = hotel_element.find_elements(By.XPATH,'//div[@class="d506630cf3"]')
        if (len(cancelacion_element) > (index + 1)):
            cancelacion = cancelacion_element[index].text
        else:
            cancelacion = ''
        
        # Detalles habitaciones
        habitaciones_element = hotel_element.find_elements(By.XPATH, '//div[@data-testid="property-card-unit-configuration"]')
        if (len(habitaciones_element) > (index + 1)):
            habitaciones = habitaciones_element[index].text
        else:
            habitaciones = ''
        
        # Puntuación
        if (len(score_element) > (index + 1)):
            score = score_element[index].text
        else:
            score = ''
            
        # Añade los datos del hotel a la lista
        hotel_data.append({"name": name, "price": price, "score": score, "address": address, "tipo_aloj": tipo_alojamiento, "cancelacion": cancelacion, "habitaciones": habitaciones})
    
    # Navega a la siguiente página de resultados de búsqueda
    offset += 25
    bookingUrl = 'https://www.booking.com/searchresults.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4AQfIAQzYAQPoAQH4AQOIAgGoAgO4At_b5aAGwAIB0gIkMjlmODkyN2EtYTczMi00Y2IxLWFiMDAtMDEwMjhiNmUzYWM52AIE4AIB&aid=397594&ss=Costa+Dorada&ssne=Costa+Dorada&ssne_untouched=Costa+Dorada&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=1389&dest_type=region&checkin=2023-07-03&checkout=2023-07-09&group_adults=2&no_rooms=1&group_children=2&age=4&age=8&sb_travel_purpose=leisure&offset='+ str(offset)
    driver.get(bookingUrl)
    
    # Espere a que cargue la siguiente página
    time.sleep(1)

# Cierre el browser
driver.quit()

# Crea el pandas dataframe con los datos del hotel
hotel_df = pd.DataFrame(hotel_data)

# Imprime el dataframe
hotel_df

hotel_df.to_csv('hoteles_booking_costadorada.csv')




In [65]:
df = pd.read_csv('hoteles_booking_costadorada.csv')

df

,Unnamed: 0,name,price,score,address,tipo_aloj,cancelacion,habitaciones
0,0,Alannia Costa Dorada,€ 936,"7,9",Platja de l’Almadrava,Bungalow de 3 dormitorios,Cancelación gratis,Bungalow entero • 3 dormitorios • 1 sala de es...
1,1,Augustus,€ 1.460,"8,0",Cambrils,Habitación Cuádruple (2 adultos + 2 niños),Cancelación gratis,Apartamento entero • 2 dormitorios • 1 sala de...
2,2,Hotel Best Terramarina,€ 1.761,"8,3",La Pineda,Habitación Familiar (2 adultos y 2 niños),Cancelación gratis - Sin pago por adelantado,Bungalow entero • 2 dormitorios • 1 sala de es...
3,3,Golden Beach Apartamentos,€ 1.319,"9,0",Sant Carles de la Ràpita,Apartamento de 2 dormitorios,Cancelación gratis,Tienda
4,4,Hotel Best Cambrils,€ 1.322,"7,6",Cambrils,Habitación Doble (2 adultos + 2 niños),Cancelación gratis - Sin pago por adelantado,Villa entera • 5 dormitorios • 1 baño • 1 coci...
...,...,...,...,...,...,...,...,...
1073,1073,Aquamar Planet Costa Dorada ONLY FAMILIES,€ 1.686,NaN,Salou,Apartamento,Cancelación gratis,Apartamento entero • 3 dormitorios • 1 sala de...
1074,1074,Esquirol 3 D - ONLY FAMILIES,€ 1.260,NaN,Cambrils,Apartamento de 3 dormitorios,Cancelación gratis,Apartamento entero • 3 dormitorios • 2 baños •...
1075,1075,HOmes & GO Cat 92 Bloq III-2,€ 1.116,NaN,"Centro de Salou, Salou",Apartamento de 2 dormitorios,NaN,Apartamento entero • 2 dormitorios • 1 baño • ...
1076,1076,2 Tus vacaciones ideales en TARRAGONA,€ 744,NaN,"Centro histórico, Tarragona",Apartamento de 1 dormitorio,NaN,Apartamento entero • 1 dormitorio • 1 baño
